In [53]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import gc

In [54]:
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
pd.set_option('display.max_rows', None) ## 모든 열을 출력한다.

In [55]:
train = pd.read_pickle("../data/train_agg.pkl",compression = "gzip")
test = pd.read_pickle("../data/test_agg.pkl",compression = "gzip")

### 데이터 
 ## num 수치형
   * std 표준편차
   * mean 평균
   * max 최대
   * min 최소
   * last 마지막값
 ## 범주형
   * count 개수
   * last 마지막 값
   * nunique 고유한 값들의 개수


In [56]:
train.columns

Index(['P_2_mean', 'P_2_std', 'P_2_min', 'P_2_max', 'P_2_last', 'D_39_mean',
       'D_39_std', 'D_39_min', 'D_39_max', 'D_39_last',
       ...
       'D_64_count', 'D_64_last', 'D_64_nunique', 'D_66_count', 'D_66_last',
       'D_66_nunique', 'D_68_count', 'D_68_last', 'D_68_nunique', 'target'],
      dtype='object', length=919)

In [57]:
train.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,S_11_last,D_62_mean,D_62_std,D_62_min,D_62_max,D_62_last,D_65_mean,D_65_std,D_65_min,D_65_max,D_65_last,B_16_mean,B_16_std,B_16_min,B_16_max,B_16_last,B_17_mean,B_17_std,B_17_min,B_17_max,B_17_last,B_18_mean,B_18_std,B_18_min,B_18_max,B_18_last,B_19_mean,B_19_std,B_19_min,B_19_max,B_19_last,B_20_mean,B_20_std,B_20_min,B_20_max,B_20_last,S_12_mean,S_12_std,S_12_min,S_12_max,S_12_last,R_6_mean,R_6_std,R_6_min,R_6_max,R_6_last,S_13_mean,S_13_std,S_13_min,S_13_max,S_13_last,B_21_mean,B_21_std,B_21_min,B_21_max,B_21_last,D_69_mean,D_69_std,D_69_min,D_69_max,D_69_last,B_22_mean,B_22_std,B_22_min,B_22_max,B_22_last,D_70_mean,D_70_std,D_70_min,D_70_max,D_70_last,D_71_mean,D_71_std,D_71_min,D_71_max,D_71_last,D_72_mean,D_72_std,D_72_min,D_72_max,D_72_last,S_15_mean,S_15_std,S_15_min,S_15_max,S_15_last,B_23_mean,B_23_std,B_23_min,B_23_max,B_23_last,D_73_mean,D_73_std,D_73_min,D_73_max,D_73_last,P_4_mean,P_4_std,P_4_min,P_4_max,P_4_last,D_74_mean,D_74_std,D_74_min,D_74_max,D_74_last,D_75_mean,D_75_std,D_75_min,D_75_max,D_75_last,D_76_mean,D_76_std,D_76_min,D_76_max,D_76_last,B_24_mean,B_24_std,B_24_min,B_24_max,B_24_last,R_7_mean,R_7_std,R_7_min,R_7_max,R_7_last,D_77_mean,D_77_std,D_77_min,D_77_max,D_77_last,B_25_mean,B_25_std,B_25_min,B_25_max,B_25_last,B_26_mean,B_26_std,B_26_min,B_26_max,B_26_last,D_78_mean,D_78_std,D_78_min,D_78_max,D_78_last,D_79_mean,D_79_std,D_79_min,D_79_max,D_79_last,R_8_mean,R_8_std,R_8_min,R_8_max,R_8_last,R_9_mean,R_9_std,R_9_min,R_9_max,R_9_last,S_16_mean,S_16_std,S_16_min,S_16_max,S_16_last,D_80_mean,D_80_std,D_80_min,D_80_max,D_80_last,R_10_mean,R_10_std,R_10_min,R_10_max,R_10_last,R_11_mean,R_11_std,R_11_min,R_11_max,R_11_last,B_27_mean,B_27_std,B_27_min,B_27_max,B_27_last,D_81_mean,D_81_std,D_81_min,D_81_max,D_81_las

In [58]:
test.head()

,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,D_39_last,B_1_mean,B_1_std,B_1_min,B_1_max,B_1_last,B_2_mean,B_2_std,B_2_min,B_2_max,B_2_last,R_1_mean,R_1_std,R_1_min,R_1_max,R_1_last,S_3_mean,S_3_std,S_3_min,S_3_max,S_3_last,D_41_mean,D_41_std,D_41_min,D_41_max,D_41_last,B_3_mean,B_3_std,B_3_min,B_3_max,B_3_last,D_42_mean,D_42_std,D_42_min,D_42_max,D_42_last,D_43_mean,D_43_std,D_43_min,D_43_max,D_43_last,D_44_mean,D_44_std,D_44_min,D_44_max,D_44_last,B_4_mean,B_4_std,B_4_min,B_4_max,B_4_last,D_45_mean,D_45_std,D_45_min,D_45_max,D_45_last,B_5_mean,B_5_std,B_5_min,B_5_max,B_5_last,R_2_mean,R_2_std,R_2_min,R_2_max,R_2_last,D_46_mean,D_46_std,D_46_min,D_46_max,D_46_last,D_47_mean,D_47_std,D_47_min,D_47_max,D_47_last,D_48_mean,D_48_std,D_48_min,D_48_max,D_48_last,D_49_mean,D_49_std,D_49_min,D_49_max,D_49_last,B_6_mean,B_6_std,B_6_min,B_6_max,B_6_last,B_7_mean,B_7_std,B_7_min,B_7_max,B_7_last,B_8_mean,B_8_std,B_8_min,B_8_max,B_8_last,D_50_mean,D_50_std,D_50_min,D_50_max,D_50_last,D_51_mean,D_51_std,D_51_min,D_51_max,D_51_last,B_9_mean,B_9_std,B_9_min,B_9_max,B_9_last,R_3_mean,R_3_std,R_3_min,R_3_max,R_3_last,D_52_mean,D_52_std,D_52_min,D_52_max,D_52_last,P_3_mean,P_3_std,P_3_min,P_3_max,P_3_last,B_10_mean,B_10_std,B_10_min,B_10_max,B_10_last,D_53_mean,D_53_std,D_53_min,D_53_max,D_53_last,S_5_mean,S_5_std,S_5_min,S_5_max,S_5_last,B_11_mean,B_11_std,B_11_min,B_11_max,B_11_last,S_6_mean,S_6_std,S_6_min,S_6_max,S_6_last,D_54_mean,D_54_std,D_54_min,D_54_max,D_54_last,R_4_mean,R_4_std,R_4_min,R_4_max,R_4_last,S_7_mean,S_7_std,S_7_min,S_7_max,S_7_last,B_12_mean,B_12_std,B_12_min,B_12_max,B_12_last,S_8_mean,S_8_std,S_8_min,S_8_max,S_8_last,D_55_mean,D_55_std,D_55_min,D_55_max,D_55_last,D_56_mean,D_56_std,D_56_min,D_56_max,D_56_last,B_13_mean,B_13_std,B_13_min,B_13_max,B_13_last,R_5_mean,R_5_std,R_5_min,R_5_max,R_5_last,D_58_mean,D_58_std,D_58_min,D_58_max,D_58_last,S_9_mean,S_9_std,S_9_min,S_9_max,S_9_last,B_14_mean,B_14_std,B_14_min,B_14_max,B_14_last,D_59_mean,D_59_std,D_59_min,D_59_max,D_59_last,D_60_mean,D_60_std,D_60_min,D_60_max,D_60_last,D_61_mean,D_61_std,D_61_min,D_61_max,D_61_last,B_15_mean,B_15_std,B_15_min,B_15_max,B_15_last,S_11_mean,S_11_std,S_11_min,S_11_max,S_11_last,D_62_mean,D_62_std,D_62_min,D_62_max,D_62_last,D_65_mean,D_65_std,D_65_min,D_65_max,D_65_last,B_16_mean,B_16_std,B_16_min,B_16_max,B_16_last,B_17_mean,B_17_std,B_17_min,B_17_max,B_17_last,B_18_mean,B_18_std,B_18_min,B_18_max,B_18_last,B_19_mean,B_19_std,B_19_min,B_19_max,B_19_last,B_20_mean,B_20_std,B_20_min,B_20_max,B_20_last,S_12_mean,S_12_std,S_12_min,S_12_max,S_12_last,R_6_mean,R_6_std,R_6_min,R_6_max,R_6_last,S_13_mean,S_13_std,S_13_min,S_13_max,S_13_last,B_21_mean,B_21_std,B_21_min,B_21_max,B_21_last,D_69_mean,D_69_std,D_69_min,D_69_max,D_69_last,B_22_mean,B_22_std,B_22_min,B_22_max,B_22_last,D_70_mean,D_70_std,D_70_min,D_70_max,D_70_last,D_71_mean,D_71_std,D_71_min,D_71_max,D_71_last,D_72_mean,D_72_std,D_72_min,D_72_max,D_72_last,S_15_mean,S_15_std,S_15_min,S_15_max,S_15_last,B_23_mean,B_23_std,B_23_min,B_23_max,B_23_last,D_73_mean,D_73_std,D_73_min,D_73_max,D_73_last,P_4_mean,P_4_std,P_4_min,P_4_max,P_4_last,D_74_mean,D_74_std,D_74_min,D_74_max,D_74_last,D_75_mean,D_75_std,D_75_min,D_75_max,D_75_last,D_76_mean,D_76_std,D_76_min,D_76_max,D_76_last,B_24_mean,B_24_std,B_24_min,B_24_max,B_24_last,R_7_mean,R_7_std,R_7_min,R_7_max,R_7_last,D_77_mean,D_77_std,D_77_min,D_77_max,D_77_last,B_25_mean,B_25_std,B_25_min,B_25_max,B_25_last,B_26_mean,B_26_std,B_26_min,B_26_max,B_26_last,D_78_mean,D_78_std,D_78_min,D_78_max,D_78_last,D_79_mean,D_79_std,D_79_min,D_79_max,D_79_last,R_8_mean,R_8_std,R_8_min,R_8_max,R_8_last,R_9_mean,R_9_std,R_9_min,R_9_max,R_9_last,S_16_mean,S_16_std,S_16_min,S_16_max,S_16_last,D_80_mean,D_80_std,D_80_min,D_80_max,D_80_last,R_10_mean,R_10_std,R_10_min,R_10_max,R_10_last,R_11_mean,R_11_std,R_11_min,R_11_max,R_11_last,B_27_mean,B_27_std,B_27_min,B_27_max,B_27_last,D_81_mean,D_81_std,D_81_min,D_81_max,D_81_las

In [59]:
train.dtypes

P_2_mean          float16
P_2_std           float64
P_2_min           float16
P_2_max           float16
P_2_last          float16
D_39_mean         float16
D_39_std          float64
D_39_min          float16
D_39_max          float16
D_39_last         float16
B_1_mean          float16
B_1_std           float64
B_1_min           float16
B_1_max           float16
B_1_last          float16
B_2_mean          float16
B_2_std           float64
B_2_min           float16
B_2_max           float16
B_2_last          float16
R_1_mean          float16
R_1_std           float64
R_1_min           float16
R_1_max           float16
R_1_last          float16
S_3_mean          float16
S_3_std           float64
S_3_min           float16
S_3_max           float16
S_3_last          float16
D_41_mean         float16
D_41_std          float64
D_41_min          float16
D_41_max          float16
D_41_last         float16
B_3_mean          float16
B_3_std           float64
B_3_min           float16
B_3_max     

In [60]:
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
label_en = LabelEncoder()
for cf in cat_features: ### 두개의 63 64 last 값 category-> 라벨 인코딩
    train[cf] = label_en.fit_transform(train[cf]).astype("float16")
    test[cf] = label_en.fit_transform(test[cf]).astype("float16")

In [61]:
"""X = train.drop("target",axis = 1)
y = train.target

X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=2022)"""

'X = train.drop("target",axis = 1)\ny = train.target\n\nX_train,X_test,y_train,y_test = train_test_split(X,y,random_state=2022)'

In [62]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=2022)
folds = []
for train_idx,test_idx in skf.split(train,train['target']):
    folds.append((train_idx,test_idx))

In [66]:
cat_models = {}

for fold in range(5):
    print(f"=============================================({fold+1})==================================================")
    train_idx, test_idx = (folds[fold])
    X_train = train.drop('target',axis = 1).iloc[train_idx].values
    X_test = train.drop('target',axis = 1).iloc[test_idx].values
    y_train = train["target"][train_idx].values
    y_test = train["target"][test_idx].values

    cat_model = CatBoostClassifier(iterations=1000, random_state=2022,nan_mode = "Min")
    cat_model.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],
                    early_stopping_rounds=35,#cat_features=cat_features,
                    verbose = 50)
    cat_models[fold] = cat_model
    print("====================================================================================================")

=============================================(1)==================================================
Learning rate set to 0.136276
0:	learn: 0.5412696	test: 0.5412989	test1: 0.5422782	best: 0.5422782 (0)	total: 206ms	remaining: 3m 25s
50:	learn: 0.2273057	test: 0.2273320	test1: 0.2312360	best: 0.2312360 (50)	total: 9.48s	remaining: 2m 56s
100:	learn: 0.2205052	test: 0.2205289	test1: 0.2260008	best: 0.2260008 (100)	total: 19.7s	remaining: 2m 55s
150:	learn: 0.2159051	test: 0.2159279	test1: 0.2238244	best: 0.2238244 (150)	total: 29.4s	remaining: 2m 45s
200:	learn: 0.2122277	test: 0.2122499	test1: 0.2226082	best: 0.2226082 (200)	total: 39.1s	remaining: 2m 35s
250:	learn: 0.2090376	test: 0.2090596	test1: 0.2219057	best: 0.2218997 (249)	total: 49.1s	remaining: 2m 26s
300:	learn: 0.2060056	test: 0.2060275	test1: 0.2212624	best: 0.2212624 (300)	total: 58.4s	remaining: 2m 15s
350:	learn: 0.2031605	test: 0.2031823	test1: 0.2211246	best: 0.2211199 (342)	total: 1m 8s	remaining: 2m 6s
400:	learn: 0.

In [75]:
sub = pd.read_csv("../data/amex-default-prediction/sample_submission.csv")
sub.head()
print(sub.shape)

(924621, 2)


In [78]:
test.shape

(924621, 918)

In [82]:
sub.iloc[:,1:]=0.0
y_pred = sub["prediction"]
for fold in range(5):
  sub.iloc[:,1:] = cat_models[fold].predict_proba(test)/5[:,1:]


<>:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
C:\Users\JeongWoo\AppData\Local\Temp\ipykernel_20992\2866816603.py:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  sub.iloc[:,1:] = cat_models[fold].predict_proba(test)/5[:,1:]
C:\Users\JeongWoo\AppData\Local\Temp\ipykernel_20992\2866816603.py:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  sub.iloc[:,1:] = cat_models[fold].predict_proba(test)/5[:,1:]
C:\Users\JeongWoo\AppData\Local\Temp\ipykernel_20992\2866816603.py:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  sub.iloc[:,1:] = cat_models[fold].predict_proba(test)/5[:,1:]
C:\Users\JeongWoo\AppData\Local\Temp\ipykernel_20992\2866816603.py:4: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  sub.iloc[:,1:] = cat_models[fold].predict_prob

TypeError: 'int' object is not subscriptable

In [ ]:
sub
.to_csv(f"submission2.csv", index=True)